In [1]:
#!pip install sentence-transformers

In [9]:
#############################################################
#BEFORE THE GUI BEGAN!!!!!!!!!!!!!!!!!
#############################################################

#############################################################
#THERE EXISTED THIS CODE FOR TEXT SUMMARIZATION
#############################################################

import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
import numpy as np
import networkx as nx
import re
from tkinter.filedialog import askopenfilename

#Function to split text into sentences by fullstop(.)
'''def read_article(text):
    
    article = text.split(". ")
    sentences =[]
    
    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]"," ").split(" "))
    
    return sentences'''

# Read the text and tokenize into sentences
def read_article(text):
    
    sentences =[]
    
    sentences = sent_tokenize(text)
    for sentence in sentences:
        sentence.replace("[^a-zA-Z0-9]"," ")

    return sentences
    

# Create vectors and calculate cosine similarity b/w two sentences
def sentence_similarity(sent1,sent2,stopwords=None):
    if stopwords is None:
        stopwords = []
    
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    
    all_words = list(set(sent1 + sent2))
    
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    
    #build the vector for the first sentence
    for w in sent1:
        if not w in stopwords:
            vector1[all_words.index(w)]+=1
    
    #build the vector for the second sentence
    for w in sent2:
        if not w in stopwords:
            vector2[all_words.index(w)]+=1
            
    return 1-cosine_distance(vector1,vector2)

# Create similarity matrix among all sentences
def build_similarity_matrix(sentences,stop_words):
    #create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences),len(sentences)))
    
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1!=idx2:
                similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],sentences[idx2],stop_words)
                
    return similarity_matrix


# Generate and return text summary
def generate_summary(text,top_n):
    
    nltk.download('stopwords')
    nltk.download('punkt')
    
    stop_words = stopwords.words('english')
    summarize_text = []
    
    # Step1: read text and tokenize
    sentences = read_article(text)
    
    # Steo2: generate similarity matrix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences,stop_words)
    
    # Step3: Rank sentences in similarirty matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    
    #Step4: sort the rank and place top sentences
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)
    
    # Step 5: get the top n number of sentences based on rank    
    for i in range(top_n):
        summarize_text.append(ranked_sentences[i][1])
    
    # Step 6 : output the summarized version
    return " ".join(summarize_text)

############################################################################################
#On this note we ended the long line of code responsible for Text Summarization
############################################################################################


############################################################################################
#Before we move on, let us initialize codes for our keywords extraction too
############################################################################################
def extract(top_n, nr_candidates, doc):
    from sklearn.feature_extraction.text import CountVectorizer

    n_gram_range = (1, 1)
    stop_words = "english"

    # Extract candidate words/phrases
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
    candidates = count.get_feature_names()

    from sentence_transformers import SentenceTransformer

    model = SentenceTransformer('distilbert-base-nli-mean-tokens')
    doc_embedding = model.encode([doc])
    candidate_embeddings = model.encode(candidates)

    from sklearn.metrics.pairwise import cosine_similarity

    top_n = 10
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

    import numpy as np
    import itertools

        # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

###########################################################################################################################
#Keywords Extraction Done
###########################################################################################################################

import tkinter as tk
import center_tk_window
import tkinter.font as TkFont
from tkinter import scrolledtext
from PIL import Image, ImageTk

win = tk.Tk()
#win.configure(background='black')
win.geometry("1300x650")
#win.configure(bg='gr)
win.title('Text Summarization and Keyword Extraction')
win.resizable(False, False)
center_tk_window.center_on_screen(win)
font = ("Helvetica", 10)
font2 = ("Segoe Boot Semilight", 7)

helv36 = TkFont.Font(family="Helvetica",size=36,weight="bold")


Frame1 = tk.Frame(win, width=1300, height=30, bg='white')
Frame1.grid_propagate(0)
Frame1.grid(row=0)    

Frame2 = tk.Frame(win, width=570, height=600, bg='white')
Frame2.grid_propagate(0)
Frame2.grid(row=1, sticky = tk.W)

Frame3 = tk.Frame(win, width=160, height=600, bg='white')
Frame3.grid_propagate(0)
Frame3.grid(row=1)    

Frame4 = tk.Frame(win, width=570, height=600, bg='white')
Frame4.grid_propagate(0)
Frame4.grid(row=1, sticky=tk.E)    

Frame5 = tk.Frame(win, width=1300, height=20, bg='black')
Frame5.grid_propagate(0)
Frame5.grid(row=2, sticky=tk.S)    


user_text = scrolledtext.ScrolledText(Frame2, width=68, height=37, wrap=tk.WORD, relief='solid')
user_text.grid(column=0, row=0, columnspan=3, padx=(5,5))

output = scrolledtext.ScrolledText(Frame4, width=68, height=37, wrap=tk.WORD, relief='solid')
output.grid(column=0, row=0, columnspan=3, padx=(5,5))

def openFile():
    global filename
    filename = askopenfilename()  
    if filename.endswith(".txt"):
        with open(filename) as f:
            text = f.read()
            user_text.insert(tk.INSERT, text)
    elif filename.endswith(".pdf"):
        import PyPDF2
         # creating a pdf file object
        pdfFileObj = open(filename, 'rb')
        # creating a pdf reader object
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        # printing number of pages in pdf file
        pages = pdfReader.numPages
        num = 0
        while num <= pages:
            print(num)
            pageObj = pdfReader.getPage(num)
            user_text.insert(tk.INSERT, pageObj.extractText())
            print(pageObj.extractText())
            num+=1
            if num == pages:
                pdfFileObj.close()
                break
   # elif filename.endswith(("doc", "docx")):
        
def clear_text():
    user_text.delete("1.0", "end")
    output.delete("1.0", "end")
    
def summarize():
    text=user_text.get("1.0","end")
    print(text)
    a = generate_summary(text, top_n=5)
    output.insert(tk.INSERT, a)

def extract_keywords():
    text = user_text.get("1.0", "end")
    b = "\n *****************************KEYWORDS*************************** \n" + str(extract(top_n=10, nr_candidates=20, doc=text)) + "\n"
    output.insert(tk.INSERT, b)

#Import
image0 = Image.open("button_import.png")
image0 = image0.resize((140, 45))
click_btn0 = ImageTk.PhotoImage(image0)
img_label0= tk.Label(image=click_btn0)
import_= tk.Button(Frame3, image=click_btn0, borderwidth=0, command=openFile)
import_.grid(column=0, row=0, pady=(0,50), padx=(10,0))

#Summarize Button
image = Image.open("button_summarize.png")
image = image.resize((140, 45))
click_btn = ImageTk.PhotoImage(image)
img_label= tk.Label(image=click_btn)
summarize= tk.Button(Frame3, image=click_btn, borderwidth=0, command=summarize)
summarize.grid(column=0, row=0, pady=(80,0), padx=(10,0))
#summarize = tk.Button(Frame3, height=10, width=40, text ="Hello", image=photo)

#Extract Keywords
image2 = Image.open("button_extract-keywords.png")
image2 = image2.resize((140, 45))
click_btn2 = ImageTk.PhotoImage(image2)
img_label2= tk.Label(image=click_btn2)
extract_keywords= tk.Button(Frame3, image=click_btn2, borderwidth=0, command=extract_keywords)
extract_keywords.grid(column=0, row=0, pady=(200,0), padx=(10,0))

#Clear
image3 = Image.open("button_clear.png")
image3 = image3.resize((140, 45))
click_btn3 = ImageTk.PhotoImage(image3)
img_label3= tk.Label(image=click_btn3)
clear= tk.Button(Frame3, image=click_btn3, borderwidth=0, command=clear_text)
clear.grid(column=0, row=0, pady=(317,0), padx=(10,0))



# Making the text read only

tk.Label(Frame5, text="Designed with love by: ", font=font2, bg='black', fg='white').grid(column=0, row=1, padx=(600,0), pady=(2,0))
win.mainloop()

In [ ]:
from docx import documents

In [12]:
!pip install python-docx

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Dee\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 437, in _error_catcher
    yield
  File "C:\Users\Dee\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\Dee\anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "C:\Users\Dee\anaconda3\lib\http\client.py", line 458, in read
    n = self.readinto(b)
  File "C:\Users\Dee\anaconda3\lib\http\client.py", line 502, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\Dee\anaconda3\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Dee\anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\Dee\anaconda3\lib\ssl.py", line 1099, in read
    return self._sslobj.read(len, buffer)
socket.timeout: The

In [3]:
!pip install PyPDF2

  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61087 sha256=6a430eab3ce88ee22d9e1ec996ccb5e1e3ae8732e2f97105cfaa11232b0be126
  Stored in directory: c:\users\dee\appdata\local\pip\cache\wheels\b1\1a\8f\a4c34be976825a2f7948d0fa40907598d69834f8ab5889de11
Successfully built PyPDF2


In [3]:
# importing required modules

import PyPDF2
 
# creating a pdf file object

pdfFileObj = open('pdftrial3.pdf', 'rb')
 
# creating a pdf reader object

pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
 
# printing number of pages in pdf file

pages = pdfReader.numPages
num = 0
while num <= pages:
    print(num)
    pageObj = pdfReader.getPage(num)
    print(pageObj.extractText())
    num+=1
    if num == pages:
        pdfFileObj.close()
        break
# creating a page object


 
# extracting text from page

#print(pageObj.extractText())
 
# closing the pdf file object
#pdfFileObj.close()

0
NAME
 
ISHOLA FOLUKE RACHEAL
 
 
MATRIC NUMBER
 
17/77JD085
 
 
DEPARTMENT
 
SOCIOLOGY
 
 
COURSE
 
SOC 
407
 
 
COURSE TITLE
 
URBANIZATION AND LABOUR MIGRATION 1
 
 
LECTURER IN CHARGE
 
DR RAJI
 
 
ASSIGNMENT
 
DISCUSS IN DETAILS THE POSITIVE EFFECTS OF MIGRATION ON THE PLACE OF 
ORIGIN.
 

1
I
SHOLA FOLUKE RACHEAL
 
 
17/77JD085
 
ISHOLA FOLUKE R
ACHEAL
 
 
17/77JD085
 
Migration decisions are influenced by d
iffering factors
 
that "push" migrants away from their home 
areas and "pull" them toward the receiving areas. It is possible that these differences are 
economically related. Such include variations in income and educational opportunity; political 
differences in
 
religious freedom; or family differences in geographic location.
 
Individuals' inclination to migrate is influenced by a variety of factors, some of which are favorable, 
such as differentials in wages or returns to migration. Among these include transporta
tion expenses, 
the effort expended in seeking for acc